In [1]:
# Importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
from matplotlib import gridspec
import matplotlib
from scipy.optimize import curve_fit

In [2]:
density = 1026 #Water density - kg/m3
c_p = 3850 #Specific heat of salt water - J/kgC
seconds = 7*24*60*60 #Time in a week - seconds
mlotst_equator = 21.95173968 # Mixed layer depth at equator - m

In [3]:
def white_noise(sigma):
    """
    Define white noise using a normal distribution
    
    """
    mean = 0
    std = sigma
    
    return np.random.normal(mean, std)

In [4]:
def lag_regress(params, years, lagint):
    lambda_a = params['lambda_a'] # Atmospheric feedback
    lambda_o = params['lambda_o']
    sigma_a = params['sigma_a']   # Variance of atmospheric forcing
    sigma_o = params['sigma_o']   # Variance of oceanic forcing
    
    Nt = 52*years # Number of timesteps - weekly [weeks]
    dt = 60*60*24*7 # Timestep - week [s]
    
    np.random.seed(4)
    
    T = np.zeros(Nt+1) # Temperature array over time [K]
    
    F_o = white_noise(sigma_o)
    Q_toa = np.zeros(Nt)
    Q_oce = np.zeros(Nt)
    for t in range(1,Nt):
        Q_oce[t] = -lambda_o*T[t] + white_noise(sigma_o) 
        Q_toa[t] = -lambda_a*T[t] + white_noise(sigma_a)
        T[t+1] = T[t] + (dt/(c_p*density*mlotst_equator))*(Q_toa[t] + Q_oce[t])
    
    lag_series = np.arange(-lagint,lagint,1)
    lagresponse_toa = np.zeros(len(lag_series))
    lagresponse_oce = np.zeros(len(lag_series))
    i = 0
    for lag in lag_series:
        if lag < 0:
            laggedtoa = Q_toa[0:lag]
            laggedoce = Q_oce[0:lag]
            T_trunc = np.roll(T, lag)[0:lag-1]
        elif lag == 0:
            laggedtoa = Q_toa
            laggedoce = Q_oce
            T_trunc = T[:-1]
        elif lag > 0:    
            laggedoce = np.roll(Q_oce,-lag)[0:-lag]
            laggedtoa = np.roll(Q_toa,-lag)[0:-lag]
            T_trunc = T[0:-lag-1]
        lagresponse_toa[i] = np.polyfit(T_trunc, laggedtoa, deg=1)[0]
        lagresponse_oce[i] = np.polyfit(T_trunc, laggedoce, deg=1)[0]
        i += 1
        
    return lagresponse_toa, lagresponse_oce, lag_series